In [101]:
import ee
import geemap
import folium

# Initialize Earth Engine
ee.Authenticate()
ee.Initialize(project = "ee-samettenborough")

In [106]:
def create_aoi(coordinates):
  aoi = ee.Geometry.Polygon(coordinates)
  return aoi

# Split the AOI into tiles
def create_tiles(aoi, grid_size_km):
    grid_size_degrees = grid_size_km / 111.32  # Approximation for latitude
    bounds = aoi.bounds().getInfo()["coordinates"][0]
    min_x, min_y = bounds[0]
    max_x, max_y = bounds[2]

    tiles = []
    x = min_x
    while x < max_x:
        y = min_y
        while y < max_y:
            chunk = ee.Geometry.Rectangle([
                x,
                y,
                min(x + grid_size_degrees, max_x),
                min(y + grid_size_degrees, max_y)
            ])
            tiles.append(chunk)
            y += grid_size_degrees
        x += grid_size_degrees

    return tiles

In [109]:
fire_1 = [[-90.1517107831, 15.5726233568],
       [-89.1726311637, 15.5726233568],
       [-89.1726311637, 16.29215598],
       [-90.1517107831, 16.29215598],
       [-90.1517107831, 15.5726233568]
          ]

aoi = create_aoi(fire_1)
tiles = create_tiles(aoi, 25)

# Create a folium map centered on Guatemala
m = folium.Map(location=[15.9, -89.6], zoom_start=8)

# Add each tile to the map
for i, tile in enumerate(tiles):
    # Get the bounds of the tile (latitude and longitude)
    bounds = tile.bounds().getInfo()["coordinates"][0]

    # Create a polygon for the tile
    folium.Polygon(
        locations=[(lat, lon) for lon, lat in bounds],
        color="blue",  # Border color
        fill=True,
        fill_color="blue",
        fill_opacity=0.2,
        weight=2
    ).add_to(m)

# Display the map
m

In [105]:
# Initialize map centered on AOI
m = folium.Map(location=[15.9, -89.6], zoom_start=8)

# Add AOI boundary to map
aoi = ee.Geometry.Polygon(fire_1)
folium.GeoJson(aoi.getInfo()).add_to(m)

# Display the map
m

In [97]:
# Define the country boundary (Guatemala)
guat = ee.FeatureCollection("FAO/GAUL/2015/level0") \
    .filter(ee.Filter.eq('ADM0_NAME', 'Guatemala')) \
    .geometry()

Map = geemap.Map()
Map.addLayer(guat, {}, "Guatemala")
Map.centerObject(guat, 7)
Map

Map(center=[15.692016609691928, -90.35759919897927], controls=(WidgetControl(options=['position', 'transparent…

In [99]:
# landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1")\
#     .filterDate("2024-01-01", "2024-02-01")\
#     .filterBounds(guat)

landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1")
filtered = (
    landsat.filterBounds(guat)
    .filterDate('2023-09-01', '2024-02-01')
    .sort("CLOUD_COVER")
)
    
Map.addLayer(landsat, {}, "Landsat")
Map

Map(center=[15.692016609691928, -90.35759919897927], controls=(WidgetControl(options=['position', 'transparent…

In [100]:
composite = ee.Algorithms.Landsat.simpleComposite(**{
    'collection': landsat,
    'asFloat': True
})

rgbVis = {'bands': ['B4', 'B3', 'B2'],
          'min': 0,
          'max': 0.3
         }

Map.addLayer(composite.clip(guat), rgbVis, "Landsat")
Map

Map(center=[15.692016609691928, -90.35759919897927], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
filtered = (
    landsat.filterBounds(guat)
    .filterDate('2023-09-01', '2024-02-01')
    .sort("CLOUD_COVER")
)
    
# Mask clouds using the QA_PIXEL band
def mask_clouds(image):
    # QA_PIXEL: Bit 5 = Cloud, Bit 7 = Cloud Shadow
    qa = image.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(1 << 5).eq(0)  # Cloud bit
    shadow_mask = qa.bitwiseAnd(1 << 7).eq(0)  # Cloud shadow bit
    return image.updateMask(cloud_mask.And(shadow_mask))

# Apply cloud masking and filter
filtered = (
    landsat.filterBounds(guat)
    .filterDate('2023-09-01', '2024-02-01')
    .map(mask_clouds)
)

# Create a median composite
composite = filtered.median()

# Visualization parameters
rgb_vis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # Red, Green, Blue bands
    'min': 0,
    'max': 20000,
}

Map.addLayer(composite.clip(guat), rgb_vis, "Landsat Composite")
Map